`Prerequisites`

To complete this exercise, you must have access to the `little_lemon` database. As an authorized user, you need to establish a connection between Python and the database via the connector API and create a cursor object

Once the connection is established, and you have a cursor object, you can select the database `little_lemon` and print the names of the tables

Tables have been used throughout the course's various sections


#### **Learning objectives:**
* Make use of connection pools to establish a connection between Python and MySQL databases.


`Scenario:`

Little Lemon’s guests need to access the database for any booking or inquiry, for example, reading the menu. - 
- Little Lemon, therefore, needs to establish a connection between the Python and MySQL databases for every operation.  

Establishing a connection every time is resource intensive and it is affecting the performance of the Little Lemon application. To improve the performance of the application, Little Lemon needs to establish a pool of database connections to facilitate the guests’ inquiries to the database.

You are tasked to help Little Lemon to create a pool of database connections using Python.



### **Task 1:**
* Create a database connection pool with three connections available for the users to connect to the database with. 
* You need to import MySQLConnectionPool class and pass the following arguments:
    * pool_name = “ll_pool_a”
    * pool_size = 3

`Expected Result`
![Pool 1 Expected](images/4_1.png)

In [4]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

db_config = {
    'database': 'little_lemon',
    'user': 'root',
    'password': ''
}

try:
    pool = MySQLConnectionPool(pool_name = "ll_pool_a",
                           pool_size = 3, #default is 5
                           **db_config) # unpack argument in dictionary
    
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)
    

The connection pool is created with a name:  ll_pool_a
The pool size is: 3


### **Task 2**
Get a connection from the database connection pool that you have created in the first task and retrieve the following columns from the Bookings table:

- BookingID
- GuestFirstName
- GuestLastName


Retrieve the required columns and put the connection back into the pool after you have completed the task.

`Expected`
![Expected 2](images/4_2.png)

In [7]:
from mysql.connector.pooling import MySQLConnectionPool

dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : ""
}

pool = MySQLConnectionPool(pool_name = "ll_pool_a",
                           pool_size = 3, #default is 5
                           **dbconfig)


# Get the connection from the connection pool "pool"
print("Getting a connection from the pool.")
connection1 = pool.get_connection()

#print("A user with connection id {} is connected to the database.".format(
#    connection1.connection_id))

#db_Info = connection1.get_server_info()
#print("MySQL server version is:", db_Info)

# Create cursor object to communicate with entire MySQL database
print("Creating a cursor object.")
cursor = connection1.cursor()

# The SQL query is:
sql_query = "SELECT BookingId, GuestFirstName, GuestLastName FROM Bookings;"

# Execute query
print("Executing the SQL query.")
cursor.execute(sql_query)

# Fetch all results that satisfy the query 
print("Fetching the query results.")
results = cursor.fetchall()

# Retrieve column names
print("Retrieving the column names.")
cols = cursor.column_names

# Print column names and records in "results" using for loop
print("Printing the results.\n")
print("""Upcoming Bookings are:\n""")
print(cols)
for result in results:
    print(result)
    
# Put the connection back to the pool    
print("\nReturning the connection back to the pool.")
connection1.close()
print("The connection is placed back into the pool for the next user to connect.")

Getting a connection from the pool.
Creating a cursor object.
Executing the SQL query.
Fetching the query results.
Retrieving the column names.
Printing the results.

Upcoming Bookings are:

('BookingId', 'GuestFirstName', 'GuestLastName')
(1, 'Anna', 'Iversen')
(2, 'Joakim', 'Iversen')
(3, 'Vanessa', 'McCarthy')
(4, 'Marcos', 'Romero')
(5, 'Hiroki', 'Yamane')
(6, 'Diana', 'Pinto')

Returning the connection back to the pool.
The connection is placed back into the pool for the next user to connect.


### **Task 3**
* The following five guests want to connect to the database:
```python
guests = ["Anna", "Marcos", "Diana", "Joakim", "Hiroki"]
```
- You only have three connections in the database connection pool. 

- Use the available connection in the pool to connect three guests and then add new connections to the pool to connect the two remaining guests. 

- Ensure that all five guests are connected to the database at the same time, by adding more connections to the pool. 


TIP: Use `add_connection` module from the pool and add a new connection if all existing connections are in use. Use try-except from Python and print the message to inform the user when connected. 

In [8]:
# Create a connection pool
from mysql.connector.pooling import MySQLConnectionPool
dbconfig = {
    "database":"little_lemon",
    "user" : "root",
    "password" : ""
}

pool = MySQLConnectionPool(pool_name = "ll_pool_a",
                           pool_size = 3, #default is 5
                           **dbconfig)

# List of the guests who want to connect to the database
guests = ["Anna", "Marcos", "Diana", "Joakim", "Hiroki"]

# To add connection to the pool, the connection must be of MySQLConnection instance 
# Also possible to create via connect module and need the import below
import mysql.connector as connector

# Assign connection to each user
for guest in guests:
    try:
        guest_connected = pool.get_connection()
        print("[{}] is connected.\n".format(guest))
    except:
        print("No more connections are available.")
        print("Adding new connection in the pool.")
        
        # Create a connection
        connection=connector.connect(user="root",password="")
        # Add the connection into the pool
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
        
        user_connected = pool.get_connection()
        print("[{}] is connected.\n".format(guest))

[Anna] is connected.

[Marcos] is connected.

[Diana] is connected.

No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

[Joakim] is connected.

No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

[Hiroki] is connected.

